In [92]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import spacy
import shutil
import os
from google.colab import drive
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [11]:
# drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
# !unzip "/content/fake-news.zip" -d "fake_news"

Archive:  /content/fake-news.zip
  inflating: fake_news/submit.csv    
  inflating: fake_news/test.csv      
  inflating: fake_news/train.csv     


In [13]:
# shutil.copytree("/content/fake_news","/content/drive/MyDrive/fake_news")

'/content/drive/MyDrive/fake_news'

In [15]:
os.listdir("fake_news")

['submit.csv', 'train.csv', 'test.csv']

In [55]:
df=pd.read_csv("fake_news/train.csv")

In [56]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [19]:
df.isna().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [57]:
df.fillna("",inplace=True)

In [58]:
df.isna().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [23]:
nlp=spacy.load("en_core_web_sm")

In [59]:
df["content"]=df["title"]+df["author"]

In [25]:
df["content"]

,content
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You FiredConsortiumnew...
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...
...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [36]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
stopwords=set(stopwords.words("english"))

In [60]:
def steming(content):
  stemmed_content=re.sub("[^a-zA-Z]"," ",content)
  stemmed_content=stemmed_content.lower()
  doc=nlp(stemmed_content)
  stemmed_content=[w.lemma_ for w in doc if not w.is_stop ]
  stemmed_content= " ".join(stemmed_content)
  return stemmed_content



In [61]:
df["content"]=df["content"].apply(steming)

In [68]:
 y=df.label.values
 x=df.content.values

In [69]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [70]:
x

array(['house dem aide   didn t comey s letter jason chaffetz tweet itdarrell lucus',
       'flynn   hillary clinton   big woman campus    breitbartdaniel j   flynn',
       'truth firedconsortiumnew com', ...,
       'macy s say receive takeover approach hudson s bay    new york timesmichael j   de la merce rachel abrams',
       'nato   russia hold parallel exercise balkansalex ansary',
       'keep f     alivedavid swanson'], dtype=object)

In [97]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=42)

In [98]:
encoder=TfidfVectorizer()

In [99]:
transformed_train_data=encoder.fit_transform(x_train)

In [100]:
transformed_train_data.toarray().shape

(16640, 22429)

In [101]:
transformed_test_data=encoder.transform(x_test)

In [102]:
gnb=GaussianNB()

In [103]:
gnb.fit(transformed_train_data.toarray(),y_train)

GaussianNB()

In [105]:
pred=gnb.predict(transformed_train_data.toarray())

In [106]:
accuracy=accuracy_score(pred,y_train)
print(f" train accuracy {accuracy}")

 train accuracy 0.9399639423076923


In [107]:
pred=gnb.predict(transformed_test_data.toarray())

In [108]:
accuracy=accuracy_score(pred,y_test)
print(f" test accuracy {accuracy}")

 test accuracy 0.7567307692307692


In [111]:
model=LogisticRegression()

In [112]:
model.fit(transformed_train_data,y_train)

LogisticRegression()

In [113]:
pred =model.predict(transformed_train_data)

In [114]:
accuracy=accuracy_score(pred,y_train)
print(f" train accuracy for the logisctic model  {accuracy}")

 train accuracy for the logisctic model  0.9838942307692308


In [117]:
pred =model.predict(transformed_test_data)

In [118]:
accuracy=accuracy_score(pred,y_test)
print(f" test accuracy for the logisctic model  {accuracy}")

 test accuracy for the logisctic model  0.9649038461538462
